In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
# Verbindung zur lokalen MongoDB (Standardport)
client = MongoClient("mongodb://localhost:27017/")

# Datenbank und Collection auswählen
db = client["thesis"]
collection = db["products"]

# Beispielhafte Dokumente abrufen
documents = list(collection.find().limit(2))

# Als DataFrame anzeigen
df = pd.DataFrame(documents)
print(df.head())

                        _id                                 name  \
0  68385ad9ebfb5afd7e144e47               Royal Canin Mini Adult   
1  68385ad9ebfb5afd7e144e48      Whiskas Knuspertaschen mit Huhn   
2  68385ad9ebfb5afd7e144e49     Vitakraft Kräcker Honig & Dinkel   
3  68385ad9ebfb5afd7e144e4a  TetraMin Hauptfutter für Zierfische   
4  68385ad9ebfb5afd7e144e4b        JR Farm Wellensittich-Schmaus   

                                         description  price            sku  \
0  Alleinfuttermittel für ausgewachsene Hunde kle...  18.99  3182550793147   
1             Snack für Katzen mit leckerer Füllung.   1.49  4008429036986   
2  Knabbersticks für Kaninchen, dreifach gebacken...   2.79  4008239252348   
3    Hauptfutter in Flockenform für alle Zierfische.   5.49  4004218745989   
4      Ausgewogenes Alleinfutter für Wellensittiche.   3.29  4024344001112   

  category        brand  stockQuantity   shelfLocation  isColdStorage  \
0  Nahrung  Royal Canin            120  Regal B1 

In [7]:
from openai import AzureOpenAI

api_key = ""

# Azure OpenAI Client konfigurieren
client = AzureOpenAI(
    api_key=api_key,  # z.B. aus Azure Portal
    api_version="2024-02-15-preview",
    azure_endpoint="https://master-thesis-resource.cognitiveservices.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15"
)

# Hilfsfunktion: Text für das Embedding vorbereiten
def build_content(doc):
    return f"{doc.get('name', '')} - {doc.get('description', '')} " \
           f"Kategorie: {doc.get('category', '')}, Marke: {doc.get('brand', '')}, " \
           f"Zieltier: {doc.get('targetAnimal', '')}, Tags: {', '.join(doc.get('tags', []))}"

# Embedding für alle Produkte erzeugen
for doc in collection.find({"contentVector": {"$exists": False}}):
    content = build_content(doc)

    try:
        embedding_response = client.embeddings.create(
            model="text-embedding-ada-002",  # DEIN DEPLOYMENT-NAME
            input=[content]
        )
        vector = embedding_response.data[0].embedding

        # Update in MongoDB (optional, für späteren Upload in Search Index)
        collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {"content": content, "contentVector": vector}}
        )
    except Exception as e:
        print(f"Fehler bei {doc["_id"]}: {e}")


In [9]:
import requests

# Konfiguration
search_service = "thesis-agent-search"  # ohne .search.windows.net
index_name = "products"

endpoint = f"https://{search_service}.search.windows.net"
url = f"{endpoint}/indexes/{index_name}/docs/index?api-version=2024-03-01-Preview"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key
}

# Produkte mit contentVector aus MongoDB abrufen
docs_with_embedding = collection.find({"contentVector": {"$exists": True}})

# Upload vorbereiten (bulk)
payload = {
    "value": []
}

for doc in docs_with_embedding:
    payload["value"].append({
        "@search.action": "mergeOrUpload",
        "id": doc.get("sku", str(doc["_id"])),  # fallback auf _id falls kein SKU vorhanden
        "name": doc.get("name", ""),
        "description": doc.get("description", ""),
        "price": doc.get("price", 0),
        "tags": doc.get("tags", []),
        "content": doc.get("content", ""),
        "contentVector": doc.get("contentVector", [])
    })

# Request senden
response = requests.post(url, headers=headers, json=payload)

# Antwort prüfen
if response.status_code == 200:
    print("Alle Dokumente erfolgreich an Azure Search Index übermittelt.")
else:
    print("Fehler beim Upload:")
    print(response.status_code, response.text)


Alle Dokumente erfolgreich an Azure Search Index übermittelt.
